In [1]:
import kagglehub
import pandas as pd
import ollama
import json
from collections import defaultdict
from wordcloud import WordCloud
import matplotlib.pyplot as plt

c:\usecase_development\quantification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
path = kagglehub.dataset_download("farukalam/yelp-restaurant-reviews")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Dev\.cache\kagglehub\datasets\farukalam\yelp-restaurant-reviews\versions\1


In [3]:
df1 = pd.read_csv("Yelp Restaurant Reviews.csv")
print(len(df1))


19896


In [4]:
df1.head()

,Yelp URL,Rating,Date,Review Text
0,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,1/22/2022,All I can say is they have very good ice cream...
1,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,6/26/2022,Nice little local place for ice cream.My favor...
2,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,8/7/2021,A delicious treat on a hot day! Staff was very...
3,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,7/28/2016,This was great service and a fun crew! I got t...
4,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,6/23/2015,This is one of my favorite places to get ice c...


In [5]:
limiting_rows =50
collation_batch_size = 10

In [6]:

# Simple analysis --> doesnt include dishes seperately 

def analyze_reviews(review):
    try:
        response = ollama.chat(
            model='qwen3',
            messages=[{
                'role': 'user',
                'content': f"""Extract sentiment rating and points from the following restaurant review. Return a JSON with these keys:
- rating (integer from 0 to 5)
- pain_points (list of single words or short phrases if there is anything negative in review, avoid sarcasm)
- positive_points (list of single words or short phrases if there is anything positive in review, avoid sarcasm)

Review:
{review}
"""
            }],
            tools=[{
                'type': 'function',
                'function': {
                    'name': 'extract_review_analysis',
                    'description': 'Extract sentiment score and key points from a restaurant review',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'rating': {'type': 'integer'},
                            'pain_points': {
                                'type': 'array',
                                'items': {'type': 'string'}
                            },
                            'positive_points': {
                                'type': 'array',
                                'items': {'type': 'string'}
                            }
                        },
                        'required': ['rating']
                    }
                }
            }]
        )

        # result = response['message']['content'] if 'message' in response else {}
        message = response['message'] if 'message'in response else None
        if message:
            tool_call = message ['tool_calls'][0] if 'tool_calls'in message else None
        else:
            result = {}
        if tool_call:
            result = tool_call['function']['arguments']
        else:
            result = {}
        
    except Exception as e:
        result = {"rating": None, "pain_points": [], "positive_points": []}
        print(f"Error processing review: {e}")
    
    return result,response


In [7]:

review_total = []
llm_response_total = []
for i,review_text in enumerate(df1["Review Text"][:limiting_rows]):
    review,response = analyze_reviews(review_text)
    review_total.append(review)
    llm_response_total.append(response)
    if (i+1)%10 == 0:
        print(f"Completed {i+1}")
    pass



Completed 10


KeyboardInterrupt: 

In [9]:
llm_response_total

[ChatResponse(model='qwen3', created_at='2025-06-12T14:51:22.7145264Z', done=True, done_reason='stop', total_duration=48178032500, load_duration=25041100, prompt_eval_count=253, prompt_eval_duration=490007400, eval_count=575, eval_duration=47661946400, message=Message(role='assistant', content='<think>\nOkay, let\'s see. The user wants me to extract the sentiment rating and points from a restaurant review. The review provided is just "D". Hmm, that\'s really short. First, I need to figure out the rating. Since the review is just a single letter, maybe it\'s a typo or maybe it\'s meant to represent a rating. Wait, the user mentioned the rating is an integer from 0 to 5. Maybe "D" stands for a specific rating? Like in some systems, D could be a grade, but that\'s not standard. Alternatively, maybe the user made a mistake and the review is incomplete. \n\nLooking at the tools provided, the function extract_review_analysis requires the rating as an integer. The review here is too short to 